# 1 Введение

## 1.1 Постановка задачи

**Заказчик** -- страховая компания  
  
**Задача** -- разработать метод преобразования данных так, чтобы по ним было сложно восстановить персональную информацию для защиты данных клиентов. При этом при преобразовании данных качество моделей машинного обучения не должно ухудшаться

## 1.2 Исходные данные

### 1.2.1 Переданные файлы
- insurance.csv

### 1.2.3 Описание данных

**Признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи  
**Целевой признак**: количество страховых выплат клиенту за последние 5 лет.

## 1.3 План обработки и анализа данных

### 1.3.1 Предобработка данных
- обзор данных

### 1.3.2 Выбор алгоритма преобразования

### 1.3.3 Проверка алгоритма преобразования

### 1.3.6 Вывод

## 1.4 Пользовательские функции

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_rows = 10
#pd.options.display.max_columns = 50
pd.options.mode.chained_assignment = None

# 2 Основная часть

## 2.1 Предобработка данных

**Обзор данных**

In [2]:
data = pd.read_csv('insurance.csv')

In [3]:
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.00,49600.00,1,0
1,0,46.00,38000.00,1,1
2,0,29.00,21000.00,0,0
3,0,21.00,41700.00,2,0
4,1,28.00,26100.00,0,0
...,...,...,...,...,...
4995,0,28.00,35700.00,2,0
4996,0,34.00,52400.00,1,0
4997,0,20.00,33900.00,2,0
4998,1,22.00,32700.00,3,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.00,5000.00,5000.00,5000.00,5000.00
mean,0.50,30.95,39916.36,1.19,0.15
std,0.50,8.44,9900.08,1.09,0.46
min,0.00,18.00,5300.00,0.00,0.00
25%,0.00,24.00,33300.00,0.00,0.00
50%,0.00,30.00,40200.00,1.00,0.00
75%,1.00,37.00,46600.00,2.00,0.00
max,1.00,65.00,79000.00,6.00,5.00


По итогам первого, общего взгляда на данные, можно сделать следующие выводы:

- набор данных состоит из 5 столбцов и 5000 строк;
- имена столбцов читабельны, но требуют приведения их к привычному виду ("under_score" нотации);
- пропуски в данных отсутствуют.
  
Необходимо сменить типы данных (в случае возможности):
- *Возраст* на *int*;
- *Зарплата* на *int*.

**Корректировка имён столбцов**

In [6]:
data.rename(columns={'Пол': 'пол',
                     'Возраст': 'возраст',
                     'Зарплата': 'зарплата',
                     'Члены семьи': 'члены_семьи',
                     'Страховые выплаты': 'страховые_выплаты',
                    }, inplace=True)
data.columns

Index(['пол', 'возраст', 'зарплата', 'члены_семьи', 'страховые_выплаты'], dtype='object')

**Преобразование типов данных столбцов**

In [7]:
data['возраст'] = data['возраст'].astype(int)
data['зарплата'] = data['зарплата'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
пол                  5000 non-null int64
возраст              5000 non-null int64
зарплата             5000 non-null int64
члены_семьи          5000 non-null int64
страховые_выплаты    5000 non-null int64
dtypes: int64(5)
memory usage: 195.4 KB


**Удаление дубликатов строк**

In [8]:
data.duplicated().sum()

153

In [9]:
data.drop_duplicates(inplace=True)
data.duplicated().sum()

0

***Промежуточный итог***  
  
На данном этапе предобработки данных:
- имена столбцов приведены к "under_score" нотации;
- типы данных столбцов *Возраст*, *Зарплата* заменены на *int*;
- удалены полные дубликаты строк.
  
Аномальные значения и пропуски в данных отсутствуют.

## 2.2 Выбор алгоритма преобразования

Перед нами стоит задача обезличивания данных переданных признаков. Важно вести преобразование таким образом, чтобы свойства данных, влияние параметров на целевой признак осталось прежними. Это позволит обучить модель ML на преобразованных данных с таким же качеством, как и на исходных данных. Преобразование данных будем осуществлять используя свойство обратимости обратимой мтарицы.  
Известно, что:
$$
\begin{gather}
    A\cdot A^{-1} = A^{-1}\cdot A = E&(1)\\
    A\cdot E=A&(2)\\
    \bigl(A\cdot B\bigr)\cdot C = A\cdot \bigl(B\cdot C\bigr)&(3)
\end{gather}
$$
Пусть $Z$ -- обратимая матрица. Тогда, учитывая (1), (2) и (3) свойство обратимости может быть описано следующим образом:
$$
\begin{gather}
    A\cdot Z = B\\
    B\cdot Z^{-1} = \bigl(A\cdot Z\bigr)\cdot Z^{-1} = A\cdot \bigl(Z\cdot Z^{-1}\bigr) = A \cdot E = A
\end{gather}
$$
Проведём эксперимент:

In [10]:
A = np.array([[ 1,  2,  3],
              [ 4,  5,  6],
              [ 7,  8,  9],
              [10, 11, 12],
              [13, 14, 15]])

In [11]:
Z = np.random.normal(size = (3,3))
np.linalg.inv(Z) # проверка на обратимость матрицы Z

array([[-0.34238658,  0.49704724,  0.43215143],
       [-0.78948466,  0.15737492, -0.17047673],
       [ 1.91454158, -0.44723717, -1.28427515]])

In [12]:
B = A @ Z
B

array([[  5.11132043,  -3.76518578,  -0.11622045],
       [ 13.68113875,  -8.44647386,   1.05292924],
       [ 22.25095708, -13.12776193,   2.22207892],
       [ 30.8207754 , -17.80905   ,   3.39122861],
       [ 39.39059373, -22.49033808,   4.5603783 ]])

In [13]:
B @ np.linalg.inv(Z)

array([[ 1.,  2.,  3.],
       [ 4.,  5.,  6.],
       [ 7.,  8.,  9.],
       [10., 11., 12.],
       [13., 14., 15.]])

То приятное чувство, когда теория и практика полностью совпали. При этом состав элементов матрицы Z не имеет значения, важно лишь то, чтоб Z была обратимой.  
  
Таким образом, алгоритм преобразования данных сведётся к следующим шагам:
- выделение признаков из набора данных;
- формирование квадратной обратимой матрицы размером, равным количеству признаков -- матрицы обезличивания;
- умножение матрицы признаков на матрицу обезличивания

## 2.3 Проверка алгоритма преобразования

Проверим наше утверждение, что качество модели ML, обученной на обезличенных данных таково же, как и качество модели ML, обученной на исходных данных

In [14]:
features = data.drop('страховые_выплаты', axis=1).values
Z =np.random.normal(size = (4,4)) 
target = data['страховые_выплаты'].values
np.linalg.inv(Z) # проверка на обратимость матрицы Z

array([[-0.24654106,  0.22426977,  0.26461866,  0.30022465],
       [ 0.41627574,  0.37209048, -0.14651143,  0.14768314],
       [ 0.11843056,  0.28219764, -0.69328592,  0.07508809],
       [ 0.04906915, -1.04318462,  0.26904606,  0.11131836]])

In [15]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('Качество модели на исходных данных:',r2_score(target, predictions))

Z_features = features @ Z
model.fit(Z_features, target)
predictions = model.predict(Z_features)
print('Качество модели на обезличенных данных:',r2_score(target, predictions))

Качество модели на исходных данных: 0.4302010046633359
Качество модели на обезличенных данных: 0.43020100466333366


***Промежуточный итог***  
  
Качество моделей, обученных на исходных и обезличенных данных, не меняется. Предложенный алгоритм обезличивания данных работоспособен

# 3 Вывод

В ходе выполнения работы было сделано следующее.  
  
По результатам этапа предобработки данных:
- имена столбцов приведены к "under_score" нотации;
- типы данных столбцов *Возраст*, *Зарплата* заменены на *int*;
- удалены полные дубликаты строк.
  
Аномальные значения и пропуски в данных отсутствовали.  
  
Затем был пребложен алгоритм обезличивания данных, основанный на свойстве обратимости обратимой матрицы. Алгоритм преобразования данных свёлся к следующим шагам:
- выделение признаков из набора данных;
- формирование квадратной обратимой матрицы размером, равным количеству признаков -- матрицы обезличивания;
- умножение матрицы признаков на матрицу обезличивания.
  
Затем была проверена работоспосодность предложенного алгоритма проверив утверждение о том, что качество модели ML, обученной на обезличенных данных таково же, как и качество модели ML, обученной на исходных данных. По результатам эксперимаента был сделан вывод о том, что качество моделей, обученных на исходных и обезличенных данных, не меняется.  
  
***Таким образом, предложенный алгоритм обезличивания данных работоспособен и может применяться Заказчиком для сокрытия персональных данных её клиентов***